# Reranking 평가

- Cross-encoder 기반 reranker 모듈을 도입하여 성능 향상 평가

In [1]:
import os, sys, builtins, logging

# repo 루트 기준으로 utils 임포트 가능하도록 경로 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.logging_setup import setup_logging

# UTF-8 파일 핸들러로 로그 초기화 (logs/{timestamp}.log)
# print를 직접 로거로 보낼 것이므로 redirect_prints는 False
setup_logging(force=True, redirect_prints=False)

# print를 로거로 보내되, 기존 콘솔 출력도 유지
_original_print = builtins.print

def print(*args, **kwargs):  # noqa: A001 (shadow builtins)
    message = " ".join(str(a) for a in args)
    logging.info(message)
    try:
        _original_print(*args, **kwargs)
    except Exception:
        # 콘솔 출력 실패 시에도 로깅은 유지
        pass

In [2]:
# OpenAI API 설정 및 라이브러리 import
from openai import OpenAI
import os, sys
from typing import List, Dict
import dotenv
import sys

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


dotenv.load_dotenv()

True

In [3]:
# openai 설정

# API 키 설정 (환경변수에서 가져오기)
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # 환경변수에 API 키를 설정해주세요
)


response_format = {
    "type": "json_schema",          # JSON 스키마 강제 모드
    "json_schema": {
        "name": "translate_result",
        "schema": {
            "type": "object",
            "properties": {
                "translated": { "type": "string" },
                "mongo_query": {
                    "type": "array",
                    "minItems": 1,
                    # 각 stage는 자유형 객체로 허용(예: {"$search": {...}}, {"$project": {...}})
                    "items": {
                        "type": "object",
                        "additionalProperties": True
                    }
                }
            },
            "required": ["translated", "mongo_query"],
            "additionalProperties": False
        }
    },
    # 스키마를 더 엄격히 따르게 함(모델이 스키마 밖 형식을 내보내지 않도록)
    "strict": True
}

def chat(
    messages: List[Dict[str, str]], 
    model: str = "gpt-4o-mini",
    response_format: dict = None,
    **kwargs
) -> str:
    """
    OpenAI GPT-4o-mini API를 사용하여 채팅 완성을 수행합니다.
    
    Args:
        messages: 대화 메시지 리스트 [{"role": "user", "content": "메시지"}]
        model: 사용할 모델명 (기본값: gpt-4o-mini)
        **kwargs: OpenAI API 매개변수들
            - temperature: 창의성 조절 (0.0-2.0, 기본값: 0.7)
            - max_tokens: 최대 토큰 수 (기본값: 1000)
            - top_p: 확률 임계값 (기본값: 0.95)
            - frequency_penalty: 빈도 페널티 (기본값: 0.0)
            - presence_penalty: 존재 페널티 (기본값: 0.0)
            - stream: 스트리밍 여부 (기본값: False)
            - 기타 OpenAI API가 지원하는 모든 매개변수
        
    Returns:
        GPT 응답 텍스트
    """
    # 기본값 설정
    default_params = {
        "temperature": 0.1,
        "max_tokens": 2048,
        "top_p": 0.95,
        "frequency_penalty": 0.0,
        "presence_penalty": 0.0
    }
    
    # 기본값과 사용자 입력 병합
    params = {**default_params, **kwargs}
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format=response_format,
            **params
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None


In [4]:
# MongoDB 연결 테스트
import json
import os
import pymongo
from pymongo import MongoClient
import dotenv

dotenv.load_dotenv()

print("=" * 50)
print("MongoDB 연결 테스트 시작")
print("=" * 50)

# 설정 파일 로드
with open('../configs/config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

# MongoDB 클라이언트 연결
mongodb_client = MongoClient(os.getenv("MONGODB_URI"))
print("🔗 MongoDB 클라이언트 연결 완료")

try:
    mongodb_client.admin.command('ping')
    print("✅ MongoDB 연결 성공!")
    
    current_db = mongodb_client[config['path']['db_name']]
    target_collection = current_db[config['path']['collection_name']]
    
    print(f"🎯 현재 데이터베이스: {config['path']['db_name']}")
    print(f"🎯 타겟 컬렉션: {config['path']['collection_name']}")
    
    # 컬렉션 통계 정보
    stats = current_db.command("collStats", config['path']['collection_name'])
    print(f"📈 문서 개수: {stats['count']:,}")
    
    print("=" * 50)
    print("✅ MongoDB 연결 테스트 완료!")
    print("=" * 50)
    
except Exception as e:
    print(f"❌ 오류: {e}")

MongoDB 연결 테스트 시작
🔗 MongoDB 클라이언트 연결 완료
✅ MongoDB 연결 성공!
🎯 현재 데이터베이스: HelloWorld-AI
🎯 타겟 컬렉션: foreigner_legalQA_v3
📈 문서 개수: 867
✅ MongoDB 연결 테스트 완료!


## 실험 2 파이프라인 기반

In [5]:
import os, sys
import pandas as pd

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

query_df = pd.read_csv("../data/helloworld_test_query_with_translation_query_20251014_000434.csv")
query_df.head()

,query,translated_query,언어,ground_truth_id,category,source,작성자,비고,소스,translated_4o_mini,mongo_query_4o_mini
0,"제 여자친구가 단속으로 출입국 보호소에 있습니다. 월급이 아직 들어오지 않았는데, ...",แฟนของผมถูกจับกุมและอยู่ที่ศูนย์กักตัวตรวจคนเข...,태국어,"['689b3a86ffd306c1cd3c09a4', '689b3a86ffd306c1...",임금체불,경기도외국인지원센터_상담사례,황예원,NaN,https://gmhr.or.kr/case/1529?sca=%EC%9E%84%EA%...,제 여자친구가 체포되어 이민 구금 센터에 있습니다. 지금 급여를 받지 못하고 있는데...,"[{'$search': {'index': 'text', 'compound': {'s..."
1,"안녕하세요, 건설 현장에서 일하고 있는 사람인데, 사장님이 월급을 안줘서 계좌가 압...",你好，我是在建筑工地工作的，但老板没有发工资，我的账户可能会被查封。遇到这种情况该怎么办呢？,중국어,"['689b3a86ffd306c1cd3c06e8', '689b3a86ffd306c1...",임금체불,경기도외국인지원센터_상담사례,황예원,"임금체불 및 ""압류방지 통장"" (=임금채권 전용통장) 관련 데이터 필요",https://gmhr.or.kr/case/1493?sca=%EC%9E%84%EA%...,"안녕하세요, 저는 건설 현장에서 일하고 있지만 사장이 급여를 지급하지 않았습니다. ...","[{'$search': {'index': 'text', 'compound': {'s..."
2,"안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 근무를 하고 이제 제 나라...","Magandang araw, ako ay isang manggagawang mula...",필리핀어 (타갈로그어),['689b3a86ffd306c1cd3c09a4'],임금체불,경기도외국인지원센터_상담사례,황예원,체당금 관련 데이터 필요,https://gmhr.or.kr/case/1667?sca=%EC%9E%84%EA%...,"안녕하세요, 저는 필리핀에서 온 근로자입니다. 5년 동안 일했는데 이제 고국으로 돌...","[{'$search': {'index': 'text', 'compound': {'s..."
3,사업장 변경 신청 이후 제가 불법체류자가 될 수 있다는 우편이 날아왔어요. 8월 2...,在申请变更工作单位之后，我收到了一封信，说我可能会变成非法滞留者。只被允许停留到8月22日，...,중국어,"['689b3a86ffd306c1cd3c0680', '689b3a86ffd306c1...",체류자격,경기도외국인지원센터_상담사례,황예원,NaN,https://gmhr.or.kr/case/1679?sca=%EC%B2%B4%EB%...,"직장 변경 신청 후, 불법 체류자가 될 수 있다는 편지를 받았습니다. 8월 22일까...","[{'$search': {'index': 'text', 'compound': {'s..."
4,"제가 중간에 퇴직을 하게 되었는데, 소득세가 체납되어 비자 연장이 안된대요. 그런데...",我中途离职了，但是因为拖欠所得税，签证无法延期。可是我听不懂相关的通知内容。,중국어,"['689b3a86ffd306c1cd3c08f8', '689b3a86ffd306c1...",체류자격,경기도외국인지원센터_상담사례,황예원,NaN,https://gmhr.or.kr/case/1703?sca=%EC%B2%B4%EB%...,"저는 중간에 퇴사했지만, 소득세 체납 때문에 비자를 연장할 수 없습니다. 관련 통지...","[{'$search': {'index': 'text', 'compound': {'s..."


In [6]:
# Reranking 모델 설정
from Azure.reranking_model import ChatModel
from dotenv import load_dotenv

load_dotenv()
with open('../configs/config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)
    
# Reranking 모델 인스턴스 생성
reranking_chat_model = ChatModel(config)

print("Reranking 기반 하이브리드 검색 모델 설정 완료")
print(f"Reranker 모델: {config['reranker_config']['model']}")
print(f"후보 문서 수: {config['reranker_config']['numCandidates']}")
print(f"최종 반환 문서 수: {config['chat_config']['top_k']}")

print("쿼리 기반 하이브리드 검색 모델 설정 완료")

# 키워드 기반 검색 함수
def get_query_model_response_with_docs(query_text, mongo_query):
    """
    키워드 기반 하이브리드 검색 모델로부터 답변을 생성하고 검색된 문서들의 인덱스를 반환
    """
    try:
        # 빈 대화 히스토리로 시작
        conversation_history = []
        
        # 키워드 기반 모델 답변 생성
        response = reranking_chat_model.generate_ai_response(
            conversation_history, 
            query_text, 
            target_collection, 
            mongo_query=mongo_query
        )
        
        return {
            "answer": response["answer"],
            "retrieved_doc_ids": response["retrieved_doc_ids"],
            "retrieved_docs": response["retrieved_docs"]
        }
        
    except Exception as e:
        print(f"오류 발생: {e}")
        return {
            "answer": "",
            "retrieved_doc_ids": [],
            "retrieved_docs": []
        }

print("쿼리 기반 검색 함수 정의 완료")


c:\Users\hwyew\Downloads\HelloWorld-AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Reranking 기반 하이브리드 검색 모델 설정 완료
Reranker 모델: Dongjin-kr/ko-reranker
후보 문서 수: 30
최종 반환 문서 수: 20
쿼리 기반 하이브리드 검색 모델 설정 완료
쿼리 기반 검색 함수 정의 완료


In [7]:
# Retrieval Correctness 계산 함수
def calculate_retrieval_correctness(retrieved_doc_ids, ground_truth_ids):
    """
    검색된 문서 ID들과 ground truth ID들을 비교하여 correctness 계산
    """
    if not retrieved_doc_ids or not ground_truth_ids:
        return 0
    
    # ground_truth_ids가 문자열 리스트인 경우 처리
    if isinstance(ground_truth_ids, str):
        try:
            # 문자열을 리스트로 변환 (예: "['id1', 'id2']" -> ['id1', 'id2'])
            import ast
            ground_truth_list = ast.literal_eval(ground_truth_ids)
        except:
            ground_truth_list = [ground_truth_ids]
    else:
        ground_truth_list = ground_truth_ids
    
    # 검색된 문서 중 하나라도 ground truth에 있으면 1, 아니면 0
    for doc_id in retrieved_doc_ids:
        if doc_id in ground_truth_list:
            return 1
    
    return 0

print("Retrieval Correctness 계산 함수 정의 완료")

# 확장된 Retrieval 메트릭 계산 함수들
def calculate_recall_at_k(retrieved_doc_ids, ground_truth_ids, k=None):
    """
    Recall@k 계산: 검색된 문서 중 관련 문서의 비율
    """
    if not retrieved_doc_ids or not ground_truth_ids:
        return 0.0
    
    # ground_truth_ids가 문자열 리스트인 경우 처리
    if isinstance(ground_truth_ids, str):
        try:
            import ast
            ground_truth_list = ast.literal_eval(ground_truth_ids)
        except:
            ground_truth_list = [ground_truth_ids]
    else:
        ground_truth_list = ground_truth_ids
    
    # k가 지정되지 않으면 검색된 문서 수만큼 사용
    if k is None:
        k = len(retrieved_doc_ids)
    
    # 상위 k개 문서만 고려
    top_k_retrieved = retrieved_doc_ids[:k]
    
    # 관련 문서 수 계산
    relevant_retrieved = sum(1 for doc_id in top_k_retrieved if doc_id in ground_truth_list)
    
    # Recall = 관련 문서 수 / 전체 관련 문서 수
    if len(ground_truth_list) == 0:
        return 0.0
    
    return relevant_retrieved / len(ground_truth_list)


def calculate_all_metrics(retrieved_doc_ids, ground_truth_ids, k=None):
    """
    모든 메트릭을 한 번에 계산
    """
    return {
        "correctness": calculate_retrieval_correctness(retrieved_doc_ids, ground_truth_ids),
        "recall_at_k": calculate_recall_at_k(retrieved_doc_ids, ground_truth_ids, k)
    }

print("확장된 Retrieval 메트릭 계산 함수들 정의 완료")


Retrieval Correctness 계산 함수 정의 완료
확장된 Retrieval 메트릭 계산 함수들 정의 완료


In [8]:
# 안전한 mongo_query 파싱 및 평가 루프 대체
import json, ast, time
from tqdm import tqdm

def parse_mongo_query(query_raw):
    # 이미 리스트[dict]
    if isinstance(query_raw, list):
        return query_raw
    # 문자열이면 ast 우선 → json → 마지막으로 단순치환 후 json
    if isinstance(query_raw, str):
        for parser in (ast.literal_eval, json.loads):
            try:
                return parser(query_raw)
            except Exception:
                pass
        # 단순 따옴표 치환 시도 (가능한 경우에만)
        try:
            sanitized = query_raw.replace("'", '"')
            return json.loads(sanitized)
        except Exception as e:
            raise e
    raise ValueError("Unsupported mongo_query type: {}".format(type(query_raw)))

# 평가 실행 (기존 변수들 재사용)
baseline_results = []
retrieved_doc_ids_list = []
correctness_scores = []
recall_at_k_scores = []

print(f"총 {len(query_df)}개의 쿼리에 대해 쿼리 기반 하이브리드 검색 평가를 시작합니다...")

for idx, row in tqdm(query_df.iterrows(), total=len(query_df), desc="쿼리 하이브리드 검색 평가"): 
    query_text = row['translated_4o_mini']
    ground_truth_ids = row['ground_truth_id']

    try:
        mongo_query = parse_mongo_query(row['mongo_query_4o_mini'])
    except Exception as e:
        print(f"파싱 실패로 해당 샘플 건너뜀: {e}")
        baseline_results.append("")
        retrieved_doc_ids_list.append([])
        correctness_scores.append(0)
        recall_at_k_scores.append(0.0)
        continue

    print(f"\n[{idx+1}/{len(query_df)}] 처리 중: {query_text[:50]}...")
    print(f"사용할 쿼리: {mongo_query}")

    result = get_query_model_response_with_docs(query_text, mongo_query)

    baseline_results.append(result['answer'])
    retrieved_doc_ids_list.append(result['retrieved_doc_ids'])

    metrics = calculate_all_metrics(result['retrieved_doc_ids'], ground_truth_ids)
    correctness_scores.append(metrics['correctness'])
    recall_at_k_scores.append(metrics['recall_at_k'])

    print(f"검색된 문서 수: {len(result['retrieved_doc_ids'])}")
    print(f"Correctness: {metrics['correctness']}")
    print(f"Recall@k: {metrics['recall_at_k']:.3f}")

    time.sleep(1)

print(f"\n=== 키워드 기반 하이브리드 검색 평가 완료! ===")
print(f"평균 Correctness: {sum(correctness_scores) / len(correctness_scores):.3f}")
print(f"평균 Recall@k: {sum(recall_at_k_scores) / len(recall_at_k_scores):.3f}")


총 20개의 쿼리에 대해 쿼리 기반 하이브리드 검색 평가를 시작합니다...


쿼리 하이브리드 검색 평가:   0%|          | 0/20 [00:00<?, ?it/s]


[1/20] 처리 중: 제 여자친구가 체포되어 이민 구금 센터에 있습니다. 지금 급여를 받지 못하고 있는데, 태국...
사용할 쿼리: [{'$search': {'index': 'text', 'compound': {'should': [{'text': {'query': ['임금체불', '체불임금', '급여 미지급', '임금 청구'], 'path': ['title', 'contents'], 'fuzzy': {'maxEdits': 1, 'prefixLength': 2}}}, {'phrase': {'query': '임금 체불', 'path': 'title', 'slop': 2}}], 'minimumShouldMatch': 1}, 'highlight': {'path': ['title', 'contents']}}}, {'$project': {'_id': 1, 'title': 1, 'contents': 1, 'url': 1, 'score': {'$meta': 'searchScore'}, 'highlights': {'$meta': 'searchHighlights'}}}, {'$limit': 20}]
검색된 문서 수: 20
Correctness: 1
Recall@k: 0.786


쿼리 하이브리드 검색 평가:   5%|▌         | 1/20 [00:26<08:31, 26.94s/it]


[2/20] 처리 중: 안녕하세요, 저는 건설 현장에서 일하고 있지만 사장이 급여를 지급하지 않았습니다. 제 계좌...
사용할 쿼리: [{'$search': {'index': 'text', 'compound': {'should': [{'text': {'query': ['임금체불', '급여 미지급', '임금 청구', '계좌 압류'], 'path': ['title', 'contents'], 'fuzzy': {'maxEdits': 1, 'prefixLength': 2}}}, {'phrase': {'query': '임금 체불', 'path': 'title', 'slop': 2}}], 'minimumShouldMatch': 1}, 'highlight': {'path': ['title', 'contents']}}}, {'$project': {'_id': 1, 'title': 1, 'contents': 1, 'url': 1, 'score': {'$meta': 'searchScore'}, 'highlights': {'$meta': 'searchHighlights'}}}, {'$limit': 20}]
검색된 문서 수: 20
Correctness: 1
Recall@k: 0.500


...


[19/20] 처리 중: 해외 출국 만기 보험금을 신청하려면 어떤 자격 조건을 충족해야 하나요? 언제부터 신청할 수...
사용할 쿼리: [{'$search': {'index': 'text', 'compound': {'should': [{'text': {'query': ['해외 출국 보험금', '보험금 신청 자격', '만기 보험금', '신청 시기'], 'path': ['title', 'contents'], 'fuzzy': {'maxEdits': 1, 'prefixLength': 2}}}, {'phrase': {'query': '보험금 신청', 'path': 'title', 'slop': 2}}], 'minimumShouldMatch': 1}, 'highlight': {'path': ['title', 'contents']}}}, {'$project': {'_id': 1, 'title': 1, 'contents': 1, 'url': 1, 'score': {'$meta': 'searchScore'}, 'highlights': {'$meta': 'searchHighlights'}}}, {'$limit': 20}]
검색된 문서 수: 20
Correctness: 1
Recall@k: 0.500


쿼리 하이브리드 검색 평가:  95%|█████████▌| 19/20 [07:05<00:20, 20.60s/it]


[20/20] 처리 중: 내 고국으로 돌아갈 때, 언제부터 귀국 비용 보험을 신청할 수 있으며, 어떤 서류를 준비해...
사용할 쿼리: [{'$search': {'index': 'text', 'compound': {'should': [{'text': {'query': ['귀국 비용 보험', '보험 신청', '서류 준비', '귀국 절차'], 'path': ['title', 'contents'], 'fuzzy': {'maxEdits': 1, 'prefixLength': 2}}}, {'phrase': {'query': '귀국 비용 보험', 'path': 'title', 'slop': 2}}], 'minimumShouldMatch': 1}, 'highlight': {'path': ['title', 'contents']}}}, {'$project': {'_id': 1, 'title': 1, 'contents': 1, 'url': 1, 'score': {'$meta': 'searchScore'}, 'highlights': {'$meta': 'searchHighlights'}}}, {'$limit': 20}]
검색된 문서 수: 20
Correctness: 1
Recall@k: 1.000


쿼리 하이브리드 검색 평가: 100%|██████████| 20/20 [07:22<00:00, 22.13s/it]


=== 키워드 기반 하이브리드 검색 평가 완료! ===
평균 Correctness: 0.850
평균 Recall@k: 0.593


In [9]:
# 키워드 기반 검색 결과 저장 및 비교 분석
import datetime

# 데이터프레임에 키워드 기반 검색 결과 추가
query_df['answer'] = baseline_results
query_df['retrieved_doc_ids'] = retrieved_doc_ids_list
query_df['correctness'] = correctness_scores
query_df['recall_at_k'] = recall_at_k_scores

# 결과 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
evaluation_csv_filename = f"../data/evaluation_results_{timestamp}.csv"
query_df.to_csv(evaluation_csv_filename, index=False, encoding='utf-8')

print(f"\n키워드 기반 검색 결과 저장 완료:")
print(f"- answer: {len(baseline_results)}개")
print(f"- retrieved_doc_ids: {len(retrieved_doc_ids_list)}개") 
print(f"- correctness: {len(correctness_scores)}개")
print(f"- recall_at_k: {len(recall_at_k_scores)}개")
print(f"- CSV 파일: {evaluation_csv_filename}")

# 평가 결과 상세 요약
total_queries = len(query_df)
correct_retrievals = sum(correctness_scores)
avg_correctness = correct_retrievals / total_queries
avg_recall = sum(recall_at_k_scores) / len(recall_at_k_scores)

print(f"\n=== 키워드 기반 하이브리드 검색 평가 결과 요약 ===")
print(f"총 쿼리 수: {total_queries}")
print(f"정확한 검색 수: {correct_retrievals}")
print(f"평균 Correctness: {avg_correctness:.3f} ({avg_correctness*100:.1f}%)")
print(f"평균 Recall@k: {avg_recall:.3f} ({avg_recall*100:.1f}%)")



키워드 기반 검색 결과 저장 완료:
- answer: 20개
- retrieved_doc_ids: 20개
- correctness: 20개
- recall_at_k: 20개
- CSV 파일: ../data/evaluation_results_20251014_003941.csv

=== 키워드 기반 하이브리드 검색 평가 결과 요약 ===
총 쿼리 수: 20
정확한 검색 수: 17
평균 Correctness: 0.850 (85.0%)
평균 Recall@k: 0.593 (59.3%)


In [10]:
# 결과 저장 및 비교 분석
import datetime

# 결과 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
reranking_csv_filename = f"../data/reranking_evaluation_results_{timestamp}.csv"
query_df.to_csv(reranking_csv_filename, index=False, encoding='utf-8')

print(f"\nReranking 평가 결과 저장 완료: {reranking_csv_filename}")


Reranking 평가 결과 저장 완료: ../data/reranking_evaluation_results_20251014_003941.csv
